In [3]:
#imports and setup
import pandas as pd
import googlemaps

GOOGLE_MAPS_API_KEY = ''
gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)

In [7]:
#load and prep data
water_systems = pd.read_csv('../datasets/raw/water/water_systems.csv').set_index('PWS ID')
violations = pd.read_csv('../datasets/raw/water/violations.csv', low_memory=False)
violations['location'] = ''

In [6]:
#Get unique addresses of all water systems that had a violation
address_table = {}

ids = list(set(violations['PWS ID'].tolist()))
for id in ids:
    address_table[id] = {
        'address' : None,
        'location' : None
    }

for row in violations.itertuples(index=True, name='Pandas'):
    _id = getattr(row, '_1')
    if address_table[_id]['address'] == None:
        _city_name = getattr(row, '_16')
        _state = getattr(row, '_3')
        _zip = getattr(row, '_13')
        address_table[_id]['address'] = _city_name + ', ' + _state + ', ' + _zip

In [ ]:
#geocode violation system addresses
for id in ids:
    address = address_table[id]['address']
    loc = gmaps.geocode(address)
    if len(loc) != 0:
        address_table[id]['location'] = loc[0]['geometry']['location']

In [ ]:
#geocode community water systems
water_system_ids = list(water_systems.index)

for _id in water_system_ids:
    if _id not in adress_table:
        record = water_systems.loc[(_id)]
        _city = getattr(record, 'City Name')
        _state = getattr(record, 'Primacy Agency')
        _zip = str(getattr(record, 'Zip Code'))
        address = str(_city) + ',' + str(_state) + ',' + _zip
        loc = gmaps.geocode(address)
        if len(loc) != 0:
            address_table[_id] = loc[0]['geometry']['location']

In [ ]:
#export to json and later convert to geojson and apply vector tools in QGIS to associated a county with each system
import json
with open('../datasets/transformed/geocoded_violations.json', 'w') as outfile:
    json.dump(address_table, outfile)